#Load Testing Student Model

In [ ]:
from transformers import AutoProcessor, AutoModelForCausalLM, AutoConfig, AutoModel, AutoTokenizer, TrainingArguments, GitVisionConfig, BertTokenizer
from google.colab import drive
drive.mount('/content/drive')

model0_path = '/content/drive/MyDrive/student_models/Cluster_0'
model1_path = '/content/drive/MyDrive/student_models/Cluster_1'
model2_path = '/content/drive/MyDrive/student_models/Cluster_2'
model3_path = '/content/drive/MyDrive/student_models/Cluster_3'
modelw_path = '/content/drive/MyDrive/student_models/Whole'

# Load the tokenizer and configuration
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
config0 = AutoConfig.from_pretrained(model0_path)
config1 = AutoConfig.from_pretrained(model1_path)
config2 = AutoConfig.from_pretrained(model2_path)
config3 = AutoConfig.from_pretrained(model3_path)
configw = AutoConfig.from_pretrained(modelw_path)

# Load the model with tokenizer and configuration
model0 = AutoModelForCausalLM.from_pretrained(model0_path).cuda()
model1 = AutoModelForCausalLM.from_pretrained(model1_path).cuda()
model2 = AutoModelForCausalLM.from_pretrained(model2_path).cuda()
model3 = AutoModelForCausalLM.from_pretrained(model3_path).cuda()
modelw = AutoModelForCausalLM.from_pretrained(model3_path).cuda()
processor = AutoProcessor.from_pretrained("microsoft/git-base-vatex")

Mounted at /content/drive


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/440 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

#Process Video

In [ ]:
! pip install av

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 15.8 MB/s eta 0:00:00


In [ ]:
import os
import av
import torch
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

video_directory = '/content/drive/MyDrive/datasets/MSRVTT/videos/all'

def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder and frames.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded and frames of shape (num_frames, new_height, new_width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]


    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)

    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    '''
    Sample a given number of frame indices from the video.
    Args:
        clip_len (`int`): Total number of frames to sample.
        frame_sample_rate (`int`): Sample every n-th frame.
        seg_len (`int`): Maximum allowed index of sample's last frame.
    Returns:
        indices (`List[int]`): List of sampled frame indices
    '''
    converted_len = int(clip_len * frame_sample_rate)
    #print('con_len:', converted_len, 'seg_len:', seg_len)
    if converted_len >= seg_len:
      return [-1]
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices

In [ ]:
video_path = os.path.join(video_directory, "video3.mp4")

container = av.open(video_path)
# sample frames
num_frames = model0.config.num_image_with_embedding
indices = sample_frame_indices(
    clip_len=num_frames, frame_sample_rate=4, seg_len=container.streams.video[0].frames
)
frames = read_video_pyav(container, indices)

#Set as Input & Inference

In [7]:
inputs = processor(images=list(frames), return_tensors="pt").to(device)
pixel_values = inputs.pixel_values
generated_ids0 = model0.generate(pixel_values=pixel_values)
generated_ids1 = model1.generate(pixel_values=pixel_values)
generated_ids2 = model2.generate(pixel_values=pixel_values)
generated_ids3 = model3.generate(pixel_values=pixel_values)
generated_idsw = modelw.generate(pixel_values=pixel_values)
cap0 = processor.batch_decode(generated_ids0, skip_special_tokens=True)
cap1 = processor.batch_decode(generated_ids1, skip_special_tokens=True)
cap2 = processor.batch_decode(generated_ids2, skip_special_tokens=True)
cap3 = processor.batch_decode(generated_ids3, skip_special_tokens=True)
capw = processor.batch_decode(generated_idsw, skip_special_tokens=True)
print("Generated caption - Cluster 0:", cap0)
print("Generated caption - Cluster 1:", cap1)
print("Generated caption - Cluster 2:", cap2)
print("Generated caption - Cluster 3:", cap3)
print("Generated caption - Cluster w:", capw)

/usr/local/lib/python3.10/dist-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  return torch.tensor(value)
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated caption - Cluster 0: ['a a']
Generated caption - Cluster 1: ['a']
Generated caption - Cluster 2: ['a']
Generated caption - Cluster 3: ['a a']
Generated caption - Cluster w: ['a a']


#Calculate Perplexity

In [8]:
# Get model output
with torch.no_grad():
    outputs0 = model0(generated_ids0)
    outputs1 = model1(generated_ids0)
    outputs2 = model2(generated_ids0)
    outputs3 = model3(generated_ids0)
    outputsw = modelw(generated_ids0)

# Calculate perplexity
logits0 = outputs0.logits
logits1 = outputs1.logits
logits2 = outputs2.logits
logits3 = outputs3.logits
logitsw = outputsw.logits

loss0 = torch.nn.functional.cross_entropy(logits0.view(-1, logits0.size(-1)), generated_ids0.view(-1))
loss1 = torch.nn.functional.cross_entropy(logits1.view(-1, logits1.size(-1)), generated_ids1.view(-1))
loss2 = torch.nn.functional.cross_entropy(logits2.view(-1, logits2.size(-1)), generated_ids2.view(-1))
loss3 = torch.nn.functional.cross_entropy(logits3.view(-1, logits3.size(-1)), generated_ids3.view(-1))
lossw = torch.nn.functional.cross_entropy(logitsw.view(-1, logitsw.size(-1)), generated_idsw.view(-1))

text_length0 = generated_ids0.size(1) # Get the length of the input sequence
text_length1 = generated_ids1.size(1)
text_length2 = generated_ids2.size(1)
text_length3 = generated_ids3.size(1)
text_lengthw = generated_idsw.size(1)

perplexity0 = torch.exp((1/text_length0)*loss0)
perplexity1 = torch.exp((1/text_length1)*loss1)
perplexity2 = torch.exp((1/text_length2)*loss2)
perplexity3 = torch.exp((1/text_length3)*loss3)
perplexityw = torch.exp((1/text_lengthw)*lossw)

print(f"Perplexity - Cluster 0: {perplexity0.item()}")
print(f"Perplexity - Cluster 1: {perplexity1.item()}")
print(f"Perplexity - Cluster 2: {perplexity2.item()}")
print(f"Perplexity - Cluster 3: {perplexity3.item()}")
print(f"Perplexity - Whole:     {perplexityw.item()}")

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Perplexity - Cluster 0: 1.4868183135986328
Perplexity - Cluster 1: 1.4889354705810547
Perplexity - Cluster 2: 1.5209195613861084
Perplexity - Cluster 3: 1.2068707942962646
Perplexity - Whole:     1.2068707942962646


#Ensemble

In [26]:
perplexity = [perplexity0.item(), perplexity1.item(), perplexity2.item(), perplexity3.item()]
cluster = [0, 1, 2, 3]
cap = [cap0, cap1, cap2, cap3]
id = [generated_ids0, generated_ids1, generated_ids2, generated_ids3]
md = [model0, model1, model2, model3]
max_index = perplexity.index(max(perplexity))
final_answer = max_index

model = md[max_index]
generated_ids = id[max_index]
generated_cap = processor.batch_decode(generated_ids, skip_special_tokens=True)

print("Students' Perplexity:", perplexity)
print("Choosen Cluster: Cluster", cluster[max_index])
print("Final answer:", generated_cap)

Students' Perplexity: [1.4868183135986328, 1.4889354705810547, 1.5209195613861084, 1.2068707942962646]
Choosen Cluster: Cluster 2
Final answer: ['a']


#Calculate Blue Score

In [40]:
import json
import spacy
from nltk.translate.bleu_score import corpus_bleu
import random
from tqdm import trange


train_jsonl_file_path = "/content/drive/MyDrive/datasets/MSRVTT/train_caption.jsonl"
datasets_train = []
# Read and process each line in the JSONL file
with open(train_jsonl_file_path, "r") as jsonl_file:
    for line in jsonl_file:

        json_data = json.loads(line)


        datasets_train.append({
            "caption": json_data["caption"],
            "clip_name": json_data["clip_name"],
            "sen_id": json_data["sen_id"]
        })

avg_score = 0
total_test_time = 500
for testing_time in trange(total_test_time):
  test_vid = None

#  test_vid = 'video3.mp4'

  vid_found = False
  while vid_found != True:
    r = random.randint(0, 4999) #4999
    print(r)
    for files in datasets_train: #we get a random video in the training set that we "did not" perform training before (0 to 4999)
      if files["clip_name"] == 'video'+str(r):
        test_vid = files["clip_name"]+'.mp4'
        vid_found = True
        break

  print("test_vid: ", test_vid)
  video_path = os.path.join(video_directory, test_vid)

  reference = []
  references = []
  candidates = []
  nlp = spacy.load('en_core_web_sm')
  for files in datasets_train:
    if files["clip_name"]+'.mp4' == test_vid:
      doc = nlp(files["caption"])
      ref_member = [token.text for token in doc]
      reference.append(ref_member)
  references.append(reference)

  container = av.open(video_path)
  # sample frames
  num_frames = model.config.num_image_with_embedding
  indices = sample_frame_indices(
      clip_len=num_frames, frame_sample_rate=4, seg_len=container.streams.video[0].frames
  )
  frames = read_video_pyav(container, indices)

  inputs = processor(images=list(frames), return_tensors="pt").to(device)
  pixel_values = inputs.pixel_values
  #########################

  #########################
  nlp = spacy.load('en_core_web_sm')
  doc = nlp(generated_cap[0])

  tokens = [token.text for token in doc]
  candidates.append(tokens)
  print()
  print("references: ", references)
  print("candidates: ", candidates)
  score = corpus_bleu(references, candidates)
  avg_score+=score
  print("score: ",score)
print("================================================")
print("total average score: ", avg_score/total_test_time)

  0%|          | 0/500 [00:00<?, ?it/s]

118
test_vid:  video118.mp4


  0%|          | 1/500 [00:03<32:58,  3.97s/it]


references:  [[['the', ' ', 'man', 'is', 'saying', 'about', 'the', 'dish', 'might', 'be', 'he', 'made', 'it', 'was', 'a', 'piece', 'of', 'meat', 'which', 'was', 'already', 'cooked'], ['a', 'person', 'is', 'with', ' ', 'a', 'large', 'piece', 'of', 'meat', 'and', 'covers', 'it', 'with', 'foil'], ['the', 'tray', 'contain', 'the', 'meat', 'with', 'gravy', 'and', 'the', 'man', 'covers', 'it', 'with', 'an', 'aluminium', 'foil'], ['a', 'man', 'in', 'a', 'blue', 'shirt', 'uncovering', 'a', 'cooked', 'piece', 'of', 'meat', 'on', 'the', 'table'], ['in', 'the', 'kitchen', 'the', 'lady', 'is', 'opening', 'the', 'grilled', 'chicken', 'item', 'and', 'their', 'features'], ['a', 'man', 'talks', 'about', 'how', 'to', 'properly', 'cook', 'a', 'chicken'], ['a', 'person', 'discribing', 'a', 'nonveg', 'dish', 'and', 'covering', 'with', 'foil'], ['a', 'chicken', 'being', 'cooked', 'in', 'a', 'pan', 'covers', 'with', 'aluminum', 'foil'], ['a', 'man', 'in', 'blue', 'shirt', 'packed', 'the', 'fried', 'chicken

  0%|          | 2/500 [00:06<25:28,  3.07s/it]

...


references:  [[['a', 'black', 'audi', 'car', 'is', 'on', 'the', 'road'], ['a', 'black', 'audi', 'hits', 'the', 'road'], ['a', 'black', 'sedan', 'driving', 'on', 'the', 'road'], ['a', 'car', 'being', 'displayed'], ['a', 'car', 'is', 'in', 'a', 'still', 'photo'], ['a', 'man', 'talks', 'about', 'cars'], ['a', 'new', 'vehicle', 'is', 'being', 'reviewed'], ['a', 'person', 'is', 'talking', 'about', 'car'], ['a', 'still', 'image', 'of', 'a', 'black', 'car'], ['a', 'still', 'image', 'of', 'an', 'audi', 'vehicle', 'is', 'being', 'shown'], ['a', 'video', 'shows', 'a', 'still', 'image', 'of', 'a', 'car'], ['clips', 'of', 'cars', 'are', 'being', 'shown', 'while', 'background', 'noise', 'and', 'people', 'talk'], ['different', 'vehicles', 'are', 'driving'], ['photos', 'of', 'sports', 'cars'], ['picture', 'of', 'an', 'audi', 'in', 'motion'], ['pictures', 'of', 'a', 'black', 'car'], ['the', 'black', 'car', 'was', 'on', 'the', 'road'], ['there', 'is', 'a', 'picture', 'of', 'a', 'black', 'car'], ['phot

100%|█████████▉| 499/500 [20:56<00:02,  2.72s/it]


references:  [[['a', 'woman', 'doing', 'some', 'thread', 'work', 'on', 'the', 'above', 'screen'], ['a', 'man', 'is', 'preparing', 'a', ' ', 'wall', 'hanger', 'with', 'a', 'thread'], ['the', 'clip', 'shows', 'a', 'woman', 'weaving', 'strands', 'of', 'fabric', 'together'], ['a', 'women', 'is', 'doing', 'some', 'craft', 'work', 'and', 'showing', 'to', 'others'], ['a', 'man', 'is', 'narrating', 'while', 'a', 'person', 'is', 'tying', 'a', 'type', 'of', 'knot'], ['one', 'man', 'can', 'make', 'something', 'with', 'thread', 'and', 'knot', 'over', 'it'], ['a', 'man', 'has', 'string', 'attached', 'to', 'two', 'pegs', 'and', 'is', 'tying', 'knots'], ['a', 'man', 'uses', 'a', 'jig', 'to', 'tie', 'together', 'a', 'braided', 'rope', 'for', 'a', 'religious', 'ceremony'], ['a', 'person', 'uses', 'a', 'small', 'holding', 'jig', 'to', 'start', 'weaving', 'some', 'materials', 'together'], ['a', 'person', 'tying', 'a', 'knot', 'and', 'explaining', 'about', 'the', 'knot'], ['a', 'guy', 'is', 'explaining',

100%|██████████| 500/500 [20:59<00:00,  2.52s/it]


references:  [[['a', 'boy', 'is', 'playing', 'mine', 'craft', 'the', 'video', 'game'], ['a', 'game', 'play', 'video', 'of', 'minecraft'], ['a', 'game', 'scene', 'with', 'fire', 'and', 'boxes'], ['a', 'girl', 'is', 'describing', 'a', 'video', 'game'], ['a', 'gun', 'is', 'shooting', 'a', 'man', 'in', 'a', 'video', 'game'], ['a', 'mine', 'craft', 'adventurer', 'wins', 'a', 'fight'], ['a', 'person', 'is', 'giving', 'game', 'instructions'], ['a', 'person', 'is', 'playing', 'a', 'video', 'game'], ['a', 'person', 'is', 'playing', 'minecraft'], ['a', 'scene', 'from', 'minecraft', 'being', 'played', 'by', 'a', 'man'], ['a', 'video', 'of', 'a', 'fight', 'in', 'minecraft'], ['a', 'woman', 'is', 'commentating', 'while', 'playing', 'minecraft'], ['a', 'woman', 'reviews', 'a', 'video', 'game'], ['person', 'is', 'playing', 'a', 'game'], ['someone', 'is', 'explaining', 'video', 'game'], ['someone', 'is', 'playing', 'a', 'game'], ['someone', 'is', 'playing', 'video', 'game'], ['someone', 'who', 'is', 